In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util , SentencesDataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
df = pd.read_csv("data/cleandata.csv")

In [ ]:
q = df['query']
desc = df['Description']
menu = df['menu']

In [ ]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(desc, convert_to_tensor=True)

In [ ]:
score = 0
for i in range(len(q)):
    clear_output(wait=True)
    query_en = model.encode(q[i],convert_to_tensor=True)
    cosine_scores = util.cos_sim(query_en, embeddings)
    all_idx = torch.topk(cosine_scores.flatten(), 5).indices
    if i in all_idx:
        score = score + 1
    print(i/len(q))
print(f'Score: {score/len(q)}')

In [ ]:
score/len(q)

Finetune

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(q, desc, test_size=0.25, random_state=42)

In [ ]:
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [ ]:
traindata = []

for i in range(len(x_train)):
    x = str(x_train.iloc[i])
    y = str(y_train.iloc[i])
    traindata.append(InputExample(texts=[x, y], label=1.0))


In [ ]:
traindata = []
for i in range(len(x_train)):
  traindata.append(InputExample(texts=[str(x_train[i]), str(y_train[i])], label = 1.0))

In [ ]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [ ]:
train_dataloader = DataLoader(traindata, shuffle=True, batch_size=16)

In [ ]:
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],epochs=20, warmup_steps=100, show_progress_bar=True)

In [ ]:
embeddings = model.encode(desc, convert_to_tensor=True)

In [ ]:
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [ ]:
score = 0
for i in range(len(x_test)):
    clear_output(wait=True)
    ans = []
    query_en = model.encode(x_test.iloc[i], convert_to_tensor=True)  # ใช้ iloc เพื่อเข้าถึงแถว
    cosine_scores = util.cos_sim(query_en, embeddings)
    all_idx = torch.topk(cosine_scores.flatten(), 5).indices
    for j in all_idx:
        ans.append(desc[int(j)]) 
    if y_test.iloc[i] in ans:
        score += 1
    print(i/len(x_test))
print(f'Score: {score/len(x_test)}')


In [ ]:
score = 0
for i in range(len(q)):
    clear_output(wait=True)
    query_en = model.encode(q[i],convert_to_tensor=True)
    cosine_scores = util.cos_sim(query_en, embeddings)
    all_idx = torch.topk(cosine_scores.flatten(), 5).indices
    if i in all_idx:
        score = score + 1
    print(i/len(q))
print(f'Score: {score/len(q)}')

In [ ]:
model.save("model")